导入'.mat'文件读取包 数据集划分，svc，数据预处理，PCA等包

In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from spectral import *
import os

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import preprocessing

读取数据并为每个样本数据进行类别标记

In [2]:
data = np.array([])
n = 0
target = np.array([])
for i in os.listdir('train/'):
    a,b = os.path.splitext(i)
    new = sio.loadmat('train/'+i)[a]
    for j in range(new.shape[0]):
        target = np.append(target,n)#为每个样本赋予标签
    data = np.append(data,new)
    n = n+1
data = np.reshape(data,(-1,200))
target = np.reshape(target,(-1,1))

数据预处理

In [3]:
scaler = preprocessing.StandardScaler().fit(data)
data = scaler.transform(data)

In [4]:
train_x,dev_x,train_y,dev_y = train_test_split(data,target,test_size=0.3)

数据PCA降维

In [5]:
pca = PCA(n_components='mle',svd_solver = 'full',copy=False)
train_x = pca.fit_transform(train_x)
dev_x = pca.transform(dev_x)

In [6]:
pca.explained_variance_ratio_.shape

(196,)

训练模型，选用svc

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {"gamma":[0.001,0.01,0.1,1,10,100],
             "C":[0.001,0.01,0.1,1,10,100]}
print("Parameters:{}".format(param_grid))

grid_search = GridSearchCV(SVC(),param_grid,cv=5) #实例化一个GridSearchCV类
grid_search.fit(train_x,train_y.ravel()) #训练，找到最优的参数，同时使用最优的参数实例化一个新的SVC estimator。

Parameters:{'gamma': [0.001, 0.01, 0.1, 1, 10, 100], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}


NameError: name 'val_x' is not defined

In [9]:
print("Test set score:{:.2f}".format(grid_search.score(dev_x,dev_y)))
print("Best parameters:{}".format(grid_search.best_params_))
print("Best score on train set:{:.2f}".format(grid_search.best_score_))

Test set score:0.95
Best parameters:{'C': 100, 'gamma': 0.01}
Best score on train set:0.94


In [10]:
clf = SVC(decision_function_shape='ovo',C = 100,gamma = 0.01)
clf.fit(train_x,train_y.ravel())
pred_y = clf.predict(dev_x)

In [11]:
clf.score(dev_x,dev_y)

0.9465832531280077

In [12]:
from sklearn.metrics import classification_report
print(classification_report(dev_y,pred_y,target_names = None))

             precision    recall  f1-score   support

        0.0       0.93      0.95      0.94       213
        1.0       0.93      0.93      0.93       554
        2.0       0.92      0.95      0.93       114
        3.0       0.99      0.99      0.99       279
        4.0       0.91      0.93      0.92       323
        5.0       0.94      0.87      0.90       195
        6.0       0.97      0.98      0.97       116
        7.0       0.99      0.99      0.99       178
        8.0       1.00      1.00      1.00       106

avg / total       0.95      0.95      0.95      2078



In [13]:
test_data = sio.loadmat('data_test_final.mat')['data_test_final']# 导入测试集

In [14]:
test_data = scaler.transform(test_data)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [15]:
test_data = pca.transform(test_data)# PCA数据降维

In [16]:
predict_y = clf.predict(test_data) # 适应训练模型预测

In [17]:
for i in range(len(predict_y)):
    if(predict_y[i] == 0):
        predict_y[i] = 2
    elif(predict_y[i] == 1):
        predict_y[i] = 3
    elif(predict_y[i] == 2):
        predict_y[i] = 5
    elif(predict_y[i] == 3):
        predict_y[i] = 6
    elif(predict_y[i] == 4):
        predict_y[i] = 8
    elif(predict_y[i] == 5):
        predict_y[i] = 10
    elif(predict_y[i] == 6):
        predict_y[i] = 11
    elif(predict_y[i] == 7):
        predict_y[i] = 12
    elif(predict_y[i] == 8):
        predict_y[i] = 14  # 修改类别标号

In [18]:
import pandas as pd
id1 = np.arange(len(predict_y))
predict_y = np.c_[id1,predict_y]
columns = ['id','y']
save_file = pd.DataFrame(columns=columns, data=predict_y)
save_file.to_csv('test_y.csv', index=False, encoding="utf-8") #将预测结果保存至csv文件